In [ ]:
import pandas as pd

In [ ]:
experiment_model_df = pd.read_excel('experiment_test_output.xlsx')
baseline_model_df = pd.read_excel('baseline_test_output.xlsx')

In [ ]:
experiment_model_df.shape, baseline_model_df.shape

In [ ]:
baseline_model_df = baseline_model_df[(baseline_model_df['LFS']>30)&(baseline_model_df['predictionrank'] ==baseline_model_df['min_predictionrank'])]
baseline_model_df['facilityid'] = baseline_model_df['facilityid'].apply(lambda x : x.split('_')[1])
baseline_model_df['facilityid'] = baseline_model_df['facilityid'].astype('int64')
baseline_model_df['masterpatientid'] = baseline_model_df['masterpatientid'].apply(lambda x : x.split('_')[1])
baseline_model_df['masterpatientid'] = baseline_model_df['masterpatientid'].astype('int64')
baseline_model_df['rth_censusdate'] = pd.to_datetime(baseline_model_df['rth_censusdate'])

In [ ]:
experiment_model_df = experiment_model_df[(experiment_model_df['LFS']>30)&(experiment_model_df['predictionrank'] ==experiment_model_df['min_predictionrank'])]
experiment_model_df['facilityid'] = experiment_model_df['facilityid'].apply(lambda x : x.split('_')[1])
experiment_model_df['facilityid'] = experiment_model_df['facilityid'].astype('int64')
experiment_model_df['masterpatientid'] = experiment_model_df['masterpatientid'].apply(lambda x : x.split('_')[1])
experiment_model_df['masterpatientid'] = experiment_model_df['masterpatientid'].astype('int64')
experiment_model_df['rth_censusdate'] = pd.to_datetime(experiment_model_df['rth_censusdate'])

experiment_model_df.shape, 

In [ ]:
baseline_df_missed = baseline_model_df[(baseline_model_df['predictionrank']>15)]
baseline_df_caught = baseline_model_df[(baseline_model_df['predictionrank']<=15)]

experiment_df_caught = experiment_model_df[experiment_model_df['predictionrank']<=15]
experiment_df_missed = experiment_model_df[experiment_model_df['predictionrank']>15]

experiment_df_caught.shape, experiment_df_missed.shape, baseline_df_missed.shape, baseline_df_caught.shape

In [ ]:
experiment_df_caught['name'] = 'experiment_caught'
experiment_df_missed['name'] = 'experiment_missed'
baseline_df_caught['name'] = 'baseline_caught'
baseline_df_missed['name'] = 'baseline_missed'

In [ ]:
baseline_df_missed.drop_duplicates(subset=['facilityid', 'masterpatientid','min_predictionrank','dateoftransfer'],inplace=True)
experiment_df_missed.drop_duplicates(subset=['facilityid', 'masterpatientid','min_predictionrank','rth_censusdate'],inplace=True)
baseline_df_missed['dateoftransfer'] = pd.to_datetime(baseline_df_missed['dateoftransfer'])
baseline_df_missed.shape, experiment_df_missed.shape

In [ ]:
both_missed_merged_df = experiment_df_missed.merge(baseline_df_missed,
                            how='inner', 
                            left_on=['facilityid', 'masterpatientid','rth_censusdate'],
                            right_on=['facilityid', 'masterpatientid','dateoftransfer'],
                            suffixes = ('_experiment', '_baseline'))
both_missed_merged_df.head()

In [ ]:
both_missed_merged_df.shape, experiment_df_missed.shape, baseline_df_missed.shape

In [ ]:
experiment_caught_baseline_missed_merged_df = experiment_df_caught.merge(baseline_df_missed,
                            how='inner', 
                            left_on=['facilityid', 'masterpatientid','rth_censusdate'],
                            right_on=['facilityid', 'masterpatientid','dateoftransfer'],
                            suffixes = ('_experiment', '_baseline'))
experiment_caught_baseline_missed_merged_df.head()

In [ ]:
baseline_df_caught['dateoftransfer'] = pd.to_datetime(baseline_df_caught['dateoftransfer'])
baseline_caught_experiment_missed_merged_df = experiment_df_missed.merge(baseline_df_caught,
                            how='inner', 
                            left_on=['facilityid', 'masterpatientid','rth_censusdate'],
                            right_on=['facilityid', 'masterpatientid','dateoftransfer'],
                            suffixes = ('_experiment', '_baseline'))
baseline_caught_experiment_missed_merged_df.head()

In [ ]:
experiment_caught_baseline_missed_merged_df.shape, baseline_caught_experiment_missed_merged_df.shape

In [ ]:
baseline_caught_experiment_caught_merged_df = experiment_df_caught.merge(baseline_df_caught,
                            how='inner', 
                            left_on=['facilityid', 'masterpatientid','rth_censusdate'],
                            right_on=['facilityid', 'masterpatientid','dateoftransfer'],
                            suffixes = ('_experiment', '_baseline'))
baseline_caught_experiment_caught_merged_df.head()

In [ ]:
baseline_caught_experiment_caught_merged_df.shape

In [ ]:
combined_df = pd.concat([baseline_caught_experiment_missed_merged_df,
                         experiment_caught_baseline_missed_merged_df,
                         both_missed_merged_df],axis=0)

combined_df.shape

In [ ]:
combined_df = pd.concat([combined_df, 
                         baseline_caught_experiment_caught_merged_df],axis=0)

In [ ]:
combined_df['name'] = 'both missed'
combined_df.loc[(combined_df['min_predictionrank_experiment']<=15)&(combined_df['min_predictionrank_baseline']<=15),'name']='both caught'
combined_df.loc[(combined_df['min_predictionrank_experiment']<=15)&(combined_df['min_predictionrank_baseline']>15),'name']='Exp < Bl'
combined_df.loc[(combined_df['min_predictionrank_experiment']>15)&(combined_df['min_predictionrank_baseline']<=15),'name']='Bl < Exp'

In [ ]:
name_group = combined_df['name'].value_counts().reset_index(name='count')
name_group

In [ ]:
name_group.iloc[0].to_dict(), name_group.iloc[1].to_dict(), name_group.iloc[2].to_dict(),name_group.iloc[3].to_dict()

In [ ]:
import plotly.express as px
fig = px.scatter(combined_df, x="min_predictionrank_baseline", y="min_predictionrank_experiment", color="name",
                title="""""")
fig.update_layout(
    title=f"""Avante | LOS>30 | Rank comparison between experiment(optuna) and baseline(hyperopt) experiment. {name_group.iloc[0].values[0]} - {name_group.iloc[0].values[1]} | {name_group.iloc[1].values[0]} - {name_group.iloc[1].values[1]} | {name_group.iloc[2].values[0]} - {name_group.iloc[2].values[1]} | {name_group.iloc[3].values[0]} - {name_group.iloc[3].values[1]}""",
    font=dict(
        family="Courier New, monospace",
        size=8,
        color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
# import os,plotly
# plotly.offline.plot(fig, filename=os.path.join('.',f'avante_los30_rank comparison between baseline and experiment.html'))